# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Wcode Class

In [5]:
class Wcode < Array
  def initialize(word = Group::Identity)
    if word != Group::Identity
      arr = Array.new
      word.flatten.factors.each do |f|
        arr += [Rseq.index(f)+1, Rseq.index(f.inverse)+1]
      end
      arr << arr.shift
      arr.each_slice(2){|a| self << a}
    end
  end
  #---
  def length(); self.flatten.size end
  def max(); self.flatten.max end
  def copy()
    wc = self.class.new
    wc.each_with_index{|a, i| wc[i] = self[i]}
    wc
  end
  def show(); self.inspect end
  #---
  def flatten()
    myarr = Array.new
    self.each{|a| myarr += a}
    myarr
  end
  #---
  def to_word()
    flt = self.flatten
    flt.unshift(flt.pop)
    gens = []
    flt.each_slice(2){|s| gens << Rseq[s[0]-1] }
    Word.new(gens)
  end
  #---
  def product(another)
    pop = self[-1].last
    self[-1][1] = another[-1][1]
    another[-1][1] = pop
    wc = self.class.new
    (self + another).each{|a| wc << a}
    wc
  end
  #---
  def devide(idxs)
    facs = [Wcode.new, Wcode.new]
    wcf = self.flatten
    (wcf[0..2*idxs[0]] + wcf[2*idxs[1]+1..-1]).each_slice(2){|s| facs[0] << s}
    arr = wcf[2*idxs[0]+1..2*idxs[1]]
    arr << arr.shift
    arr.each_slice(2){|s| facs[1] << s}
    return facs
  end
  #---
  ## Turaev cobracket for Wcodes
  def cobracket()
    combs=(0..self.size-1).to_a.combination(2)
    monomials = combs.map do |idxs|
      four = idxs.map{|i| self[i]}
      inner = four[1].select{|n| four[0].min < n and n < four[0].max}
      linking = if inner.size.even?
                  0
                elsif (four[0][0] < four[0][1]) ^ (inner[0]==four[1][0])
                  1
                else
                  -1
                end
      factors = self.devide(idxs)
      #---
      {pairs: four, linking: linking, factors: factors }
    end
    return monomials #.select{|h| h[:linking] != 0}
  end  
end; nil

In [6]:
w = Word.new(a[1], b[2], b[1].inverse)
p w.show
printf("-----\n")

wc = Wcode.new(w)
p wc, wc.size, wc.length, wc.flatten, wc.to_word.show
printf("-----\n")

wc.cobracket.each do |mono|
  p mono[:pairs]
  p mono[:linking]
  facs = mono[:factors]
  p facs.map(&:to_s).join(" \u{2227} ")
  p facs.map(&:to_word).map(&:show).join(" \u{2227} ")
  printf("---\n")
end
nil

"adB"
-----
[[3, 8], [6, 2], [4, 1]]
3
6
[3, 8, 6, 2, 4, 1]
"adB"
-----
[[3, 8], [6, 2]]
-1
"[[3, 2], [4, 1]] ∧ [[6, 8]]"
"aB ∧ d"
---
[[3, 8], [4, 1]]
-1
"[[3, 1]] ∧ [[6, 2], [4, 8]]"
"a ∧ dB"
---
[[6, 2], [4, 1]]
1
"[[3, 8], [6, 1]] ∧ [[4, 2]]"
"ad ∧ B"
---


In [7]:
ww = Word.new(a[1])^3
wc = Wcode.new(ww)
p wc.show, wc.to_word.show
printf("-----\n")

wc.cobracket.each do |mono|
  p mono[:pairs]
  p mono[:linking]
  facs = mono[:factors]
  p facs.map(&:to_s).join(" \u{2227} ")
  p facs.map(&:to_word).map(&:show).join(" \u{2227} ")
  printf("---\n")
end
nil

"[[3, 1], [3, 1], [3, 1]]"
"aaa"
-----
[[3, 1], [3, 1]]
0
"[[3, 1], [3, 1]] ∧ [[3, 1]]"
"aa ∧ a"
---
[[3, 1], [3, 1]]
0
"[[3, 1]] ∧ [[3, 1], [3, 1]]"
"a ∧ aa"
---
[[3, 1], [3, 1]]
0
"[[3, 1], [3, 1]] ∧ [[3, 1]]"
"aa ∧ a"
---


In [8]:
w2 = Word.new(a[2])
wc2 = Wcode.new(w2)

p wc.to_s + ' * ' + wc2.to_s

p wc.product(wc2)
nil

"[[3, 1], [3, 1], [3, 1]] * [[7, 5]]"
[[3, 1], [3, 1], [3, 5], [7, 1]]


In [9]:
p wc.to_word.show, wc
printf("===\n")

wc.cobracket.each do |h|
  p "pairs : #{h[:pairs]}"
  p "linking : #{h[:linking]}"
  p "factors : " + h[:factors].map(&:show).join(' , ')
  p h[:factors].map(&:to_word).map(&:show).join(" \u{2227} ")
#  p h[:factors].map(&:to_word).map(&:cyclic_reduce).map(&:show).join(" \u{2227} ")
#  prd = h[:factors].inject(&:product)
#  p prd, prd.to_word.show
  printf("-----\n")
end
nil

"caa"
[[3, 1], [3, 1], [3, 5]]
===
"pairs : [[3, 1], [3, 1]]"
"linking : 0"
"factors : [[3, 1], [3, 5]] , [[3, 1]]"
"ca ∧ a"
-----
"pairs : [[3, 1], [3, 5]]"
"linking : 0"
"factors : [[3, 5]] , [[3, 1], [3, 1]]"
"c ∧ aa"
-----
"pairs : [[3, 1], [3, 5]]"
"linking : 0"
"factors : [[3, 1], [3, 5]] , [[3, 1]]"
"ca ∧ a"
-----


# Execution

In [10]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten
samples = [
  a1,
  a1*Word.new(b[2]),
  a1*(Word.new(b[2]).inverse),
  #---
  a1^4,
  #---
  a1*(comms[2].inverse),
  a1*comms[2],
  a1*(comms[2]^3),
  a1*(a2^3),
  a1*(Word.new(a[2], b[2].inverse)^3),
  a1*comms[2]*c*comms[2]*(c.inverse),
  #---
  a1*Group.commutator(a[2], a[3]),
  a1*comms[2]*comms[3]*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ]

samples.each do |wa|
  ws = wa.show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  terms = Wcode.new(wa).cobracket.map do |h|
    coeff = (h[:linking]==1) ? '' : '(-1)'
    words = h[:factors].map(&:to_word)
    coeff + words.map(&:show).join(" \u{2227} ")
  end
  p "#{ws}  |--\u{03B4}--> #{terms.join(' + ')}"
end;nil

"a  |--δ--> "
"a.d  |--δ--> (-1)a ∧ d"
"a.D  |--δ--> (-1)a ∧ D"
"a.a.a.a  |--δ--> (-1)aaa ∧ a + (-1)aa ∧ aa + (-1)a ∧ aaa + (-1)aaa ∧ a + (-1)aa ∧ aa + (-1)aaa ∧ a"
"a.dcDC  |--δ--> (-1)acDC ∧ d + (-1)aDC ∧ dc + aC ∧ dcD + (-1)a ∧ dcDC + (-1)adDC ∧ c + (-1)adC ∧ cD + (-1)ad ∧ cDC + (-1)adcC ∧ D + (-1)adc ∧ DC + (-1)adcD ∧ C"
"a.cdCD  |--δ--> (-1)adCD ∧ c + (-1)aCD ∧ cd + (-1)aD ∧ cdC + (-1)a ∧ cdCD + (-1)acCD ∧ d + (-1)acD ∧ dC + (-1)ac ∧ dCD + (-1)acdD ∧ C + (-1)acd ∧ CD + (-1)acdC ∧ D"
"a.cdCD.cdCD.cdCD  |--δ--> (-1)adCDcdCDcdCD ∧ c + (-1)aCDcdCDcdCD ∧ cd + (-1)aDcdCDcdCD ∧ cdC + (-1)acdCDcdCD ∧ cdCD + (-1)adCDcdCD ∧ cdCDc + (-1)aCDcdCD ∧ cdCDcd + (-1)aDcdCD ∧ cdCDcdC + (-1)acdCD ∧ cdCDcdCD + (-1)adCD ∧ cdCDcdCDc + (-1)aCD ∧ cdCDcdCDcd + (-1)aD ∧ cdCDcdCDcdC + (-1)a ∧ cdCDcdCDcdCD + (-1)acCDcdCDcdCD ∧ d + (-1)acDcdCDcdCD ∧ dC + (-1)accdCDcdCD ∧ dCD + (-1)acdCDcdCD ∧ dCDc + (-1)acCDcdCD ∧ dCDcd + (-1)acDcdCD ∧ dCDcdC + (-1)accdCD ∧ dCDcdCD + (-1)acdCD ∧ dCDcdCDc + (-1)acCD ∧ dCDcdCDcd

"a.cdCD.efEF.ceCE.feFEdcDC  |--δ--> (-1)adCDefEFceCEfeFEdcDC ∧ c + (-1)aCDefEFceCEfeFEdcDC ∧ cd + (-1)aDefEFceCEfeFEdcDC ∧ cdC + (-1)aefEFceCEfeFEdcDC ∧ cdCD + (-1)afEFceCEfeFEdcDC ∧ cdCDe + (-1)aEFceCEfeFEdcDC ∧ cdCDef + (-1)aFceCEfeFEdcDC ∧ cdCDefE + (-1)aceCEfeFEdcDC ∧ cdCDefEF + (-1)aeCEfeFEdcDC ∧ cdCDefEFc + (-1)aCEfeFEdcDC ∧ cdCDefEFce + (-1)aEfeFEdcDC ∧ cdCDefEFceC + (-1)afeFEdcDC ∧ cdCDefEFceCE + (-1)aeFEdcDC ∧ cdCDefEFceCEf + (-1)aFEdcDC ∧ cdCDefEFceCEfe + (-1)aEdcDC ∧ cdCDefEFceCEfeF + (-1)adcDC ∧ cdCDefEFceCEfeFE + (-1)acDC ∧ cdCDefEFceCEfeFEd + (-1)aDC ∧ cdCDefEFceCEfeFEdc + (-1)aC ∧ cdCDefEFceCEfeFEdcD + (-1)a ∧ cdCDefEFceCEfeFEdcDC + (-1)acCDefEFceCEfeFEdcDC ∧ d + (-1)acDefEFceCEfeFEdcDC ∧ dC + (-1)acefEFceCEfeFEdcDC ∧ dCD + (-1)acfEFceCEfeFEdcDC ∧ dCDe + (-1)acEFceCEfeFEdcDC ∧ dCDef + (-1)acFceCEfeFEdcDC ∧ dCDefE + (-1)acceCEfeFEdcDC ∧ dCDefEF + (-1)aceCEfeFEdcDC ∧ dCDefEFc + (-1)acCEfeFEdcDC ∧ dCDefEFce + (-1)acEfeFEdcDC ∧ dCDefEFceC + (-1)acfeFEdcDC ∧ dCDefEFceCE + (-1

* "a |--δ--> 0"
* "a.d |--δ--> a ∧ d"
* "a.D |--δ--> 0"
* "a.a.a.a |--δ--> (-1) aa ∧ aa"
* "abA |--δ--> (-1) 1 ∧ b"
* "abAB |--δ--> 0"
* "a.dcDC |--δ--> a ∧ dcDC"
* "a.cdCD |--δ--> 0"
* "a.cdCD.cdCD.cdCD |--δ--> (-1) acdCDcdCD ∧ cdCD + (-1) acdCD ∧ cdCDcdCD"
* "a.c.c.c |--δ--> (-1) acc ∧ c + (-1) ac ∧ cc"
* "a.cD.cD.cD |--δ--> (-1) acDcD ∧ cD + (-1) acD ∧ cDcD"
* "a.cdCD.abA.cdCD.aBA |--δ--> (-1) acdCD ∧ cdCD + (-1) cdCD ∧ cdCDa"
* "a.ceCE |--δ--> aceC ∧ E + (-1) ace ∧ CE + ac ∧ C"
* "a.cdCD.efEF.ceCE.feFEdcDC |--δ--> acdCDefEFceCfeFEdcDC ∧ E + (-1) acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"

In [11]:
require 'rbplotly'

N, K = 100, 2.0

Xstd = (1..N).map{|k| k.to_f/N.to_f}

def get_trace(pair)
  d = pair[1]-pair[0]
  xa = Xstd.map{|x| d*x+pair[0]}
  ya = xa.map{|x| K*(x-pair[0])*(x-pair[1])}
  { x: xa, y: ya}
end

traces = wc.map{|pair| get_trace(pair)}
pl = Plotly::Plot.new(data: traces)
pl.show
nil

#<Plotly::Offline::HTML:0x00007f9c3a166cb8 @id="5c44f5f0-d47f-4ce3-b9c0-3f3083cda8aa", @data=[{:x=>[2.98, 2.96, 2.94, 2.92, 2.9, 2.88, 2.86, 2.84, 2.82, 2.8, 2.78, 2.76, 2.74, 2.7199999999999998, 2.7, 2.68, 2.66, 2.64, 2.62, 2.6, 2.58, 2.56, 2.54, 2.52, 2.5, 2.48, 2.46, 2.44, 2.42, 2.4, 2.38, 2.36, 2.34, 2.32, 2.3, 2.2800000000000002, 2.26, 2.24, 2.2199999999999998, 2.2, 2.18, 2.16, 2.14, 2.12, 2.1, 2.08, 2.06, 2.04, 2.02, 2.0, 1.98, 1.96, 1.94, 1.92, 1.9, 1.88, 1.86, 1.84, 1.82, 1.8, 1.78, 1.76, 1.74, 1.72, 1.7, 1.68, 1.66, 1.64, 1.62, 1.6, 1.58, 1.56, 1.54, 1.52, 1.5, 1.48, 1.46, 1.44, 1.42, 1.4, 1.38, 1.36, 1.34, 1.32, 1.3, 1.28, 1.26, 1.24, 1.22, 1.2, 1.18, 1.16, 1.14, 1.12, 1.1, 1.08, 1.06, 1.04, 1.02, 1.0], :y=>[-0.07920000000000008, -0.15680000000000013, -0.2328000000000002, -0.30720000000000025, -0.38000000000000034, -0.4512000000000004, -0.5208000000000004, -0.5888000000000004, -0.6552000000000006, -0.7200000000000005, -0.7832000000000006, -0.8448000000000007, -0.9047999999999994, -0.9632000000000007, -1.0199999999999996, -1.0751999999999995, -1.1287999999999996, -1.1807999999999996, -1.2311999999999996, -1.2799999999999998, -1.3272, -1.3727999999999998, -1.4167999999999998, -1.4592, -1.5, -1.5392000000000001, -1.5768, -1.6128, -1.6472000000000002, -1.6800000000000002, -1.7112, -1.7408000000000001, -1.7688000000000001, -1.7952000000000001, -1.8200000000000003, -1.8431999999999997, -1.8648000000000002, -1.8847999999999998, -1.9032000000000002, -1.92, -1.9351999999999998, -1.9487999999999999, -1.9607999999999999, -1.9711999999999998, -1.98, -1.9872, -1.9928, -1.9968, -1.9992, -2.0, -1.9992, -1.9968, -1.9928, -1.9872, -1.98, -1.9711999999999998, -1.9608, -1.9488, -1.9352, -1.92, -1.9032, -1.8848, -1.8648, -1.8432, -1.8199999999999998, -1.7952, -1.7688, -1.7408, -1.7112, -1.6800000000000002, -1.6472000000000002, -1.6128, -1.5768, -1.5392000000000001, -1.5, -1.4592, -1.4167999999999998, -1.3727999999999998, -1.3272, -1.2799999999999998, -1.2311999999999996, -1.1808000000000003, -1.1288000000000002, -1.0752000000000002, -1.02, -0.9632000000000001, -0.9048, -0.8448, -0.7831999999999999, -0.7199999999999999, -0.6551999999999998, -0.5887999999999998, -0.5207999999999997, -0.4512000000000004, -0.38000000000000034, -0.30720000000000025, -0.2328000000000002, -0.15680000000000013, -0.07920000000000008, -0.0]}, {:x=>[2.98, 2.96, 2.94, 2.92, 2.9, 2.88, 2.86, 2.84, 2.82, 2.8, 2.78, 2.76, 2.74, 2.7199999999999998, 2.7, 2.68, 2.66, 2.64, 2.62, 2.6, 2.58, 2.56, 2.54, 2.52, 2.5, 2.48, 2.46, 2.44, 2.42, 2.4, 2.38, 2.36, 2.34, 2.32, 2.3, 2.2800000000000002, 2.26, 2.24, 2.2199999999999998, 2.2, 2.18, 2.16, 2.14, 2.12, 2.1, 2.08, 2.06, 2.04, 2.02, 2.0, 1.98, 1.96, 1.94, 1.92, 1.9, 1.88, 1.86, 1.84, 1.82, 1.8, 1.78, 1.76, 1.74, 1.72, 1.7, 1.68, 1.66, 1.64, 1.62, 1.6, 1.58, 1.56, 1.54, 1.52, 1.5, 1.48, 1.46, 1.44, 1.42, 1.4, 1.38, 1.36, 1.34, 1.32, 1.3, 1.28, 1.26, 1.24, 1.22, 1.2, 1.18, 1.16, 1.14, 1.12, 1.1, 1.08, 1.06, 1.04, 1.02, 1.0], :y=>[-0.07920000000000008, -0.15680000000000013, -0.2328000000000002, -0.30720000000000025, -0.38000000000000034, -0.4512000000000004, -0.5208000000000004, -0.5888000000000004, -0.6552000000000006, -0.7200000000000005, -0.7832000000000006, -0.8448000000000007, -0.9047999999999994, -0.9632000000000007, -1.0199999999999996, -1.0751999999999995, -1.1287999999999996, -1.1807999999999996, -1.2311999999999996, -1.2799999999999998, -1.3272, -1.3727999999999998, -1.4167999999999998, -1.4592, -1.5, -1.5392000000000001, -1.5768, -1.6128, -1.6472000000000002, -1.6800000000000002, -1.7112, -1.7408000000000001, -1.7688000000000001, -1.7952000000000001, -1.8200000000000003, -1.8431999999999997, -1.8648000000000002, -1.8847999999999998, -1.9032000000000002, -1.92, -1.9351999999999998, -1.9487999999999999, -1.9607999999999999, -1.9711999999999998, -1.98, -1.9872, -1.9928, -1.9968, -1.9992, -2.0, -1.9992, -1.9968, -1.9928, -1.9872, -1.98, -1.9711999999999998, -1.9608, -1.9488, -1.9352, -1.92, -1.9032, -1.8848, -1.8648, -1.84

# TODO
* [ ] cobracket の計算結果が前バージョンと一致していないので、原因究明する。
どうも、Wcode に同じ数字が何度も出てくるところが変なようだ。